In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.layers import Dense

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers import ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

from sklearn.metrics import log_loss
from imutils import paths
import cv2, os


TRAIN_DIR = "./train"
MODEL_DIR = "./models"
TEST_DIR = "./Nishant"

Using TensorFlow backend.


In [2]:
### read train images
IMAGE_DIMS = (64, 64, 3)

imagePaths = sorted(list(paths.list_images(TRAIN_DIR)))
data = []
labels = []
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
 
    # extract set of class labels from the image path and update the
    # labels list
    l = imagePath.split(os.path.sep)[-2]
    labels.append(l)

#### pre- process data

In [70]:
## convert into arrays
data = np.array(data)
labels = np.array(labels)
data=preprocess_input(data)

## label encoding
le = LabelEncoder()
labels_int = le.fit_transform(labels)
labels_en = to_categorical(labels_int)

# split data into train and validation
X_train, X_valid, Y_train, Y_valid=train_test_split(data,labels_en,test_size=0.1, random_state=1234)

#### Modeling
- freeze the weights of the first 8 layers of the vgg16 network, while we retrain the subsequent layers and add new fc layers to classify

In [78]:
def vgg16_model(height, width, depth, num_classes=None, lr_rate = 0.01):

    base_model = keras.applications.VGG16(weights='imagenet', include_top=False,
                                          input_shape = (width, height, depth))
    print("Weights loaded")

    top_model = Sequential()

    top_model.add(Flatten(input_shape = base_model.output_shape[1:]))
    
    top_model.add(Dense(256, activation='relu'))
    
    top_model.add(Dropout(0.2))
    
    top_model.add(Dense(num_classes, activation='softmax'))
    
    model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

    #To set the first 8 layers to non-trainable (weights will not be updated)

    for layer in model.layers[:13]:
        layer.trainable = False

    # Learning rate is changed to 0.001
    opt = Adam(lr=lr_rate, decay=1e-6)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
num_classes = 11
batch_size = 32 
epochs = 10

# load model
model = vgg16_model(IMAGE_DIMS[1], IMAGE_DIMS[0], IMAGE_DIMS[2], num_classes)

model.summary()

Weights loaded
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0     

In [ ]:
# fit the model
model.fit(X_train, Y_train,batch_size=batch_size,epochs=epochs,
          shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

Train on 38784 samples, validate on 4310 samples
Epoch 1/10
38784/38784 [==============================] - 2629s 68ms/step - loss: 14.3054 - acc: 0.1124 - val_loss: 14.4913 - val_acc: 0.1009
Epoch 2/10
38784/38784 [==============================] - 2629s 68ms/step - loss: 14.2995 - acc: 0.1128 - val_loss: 14.4913 - val_acc: 0.1009
Epoch 3/10
38784/38784 [==============================] - 37036s 955ms/step - loss: 14.2829 - acc: 0.1139 - val_loss: 14.4913 - val_acc: 0.1009
Epoch 4/10
 7904/38784 [=====>........................] - ETA: 2:01:07 - loss: 14.2542 - acc: 0.1156